In [4]:
/// Setup MathJax and HTML helpers
@"<script src=""https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.7/MathJax.js?config=TeX-MML-AM_CHTML""></script>" |> Util.Html |> Display

let html x = { Html = x }

let h1 text = { Html = "<h1>" + text + "</h1>" }

let h2 text = { Html = "<h2>" + text + "</h2>" }

let h3 text = { Html = "<h3>" + text + "</h3>" }

let ul text = { Html = "<ul><li>" + text + "</li></ul>" }

let ul3 text = { Html = "<ul><li><h3>" + text + "</h3></li></ul>" }

let img url = { Html = "<img src=\"" + url + "\"" + " style=\"align:center\" />" }


In [5]:
/// Load Sylvester math libraries and open namespaces
#load "MathInclude.fsx"
open Sylvester
open Sylvester.CAS

<center>
    
<img src="http://fsharp.org/img/logo/fsharp512.png" />
</center>

# The Z3 SMT solver and functional programming

# Allister Beharry

# Symbolic Logic and SAT/SMT Solvers

# Symbolic Logic
* A mathematical model of deductive thought
* Consider algebraic symbol-manipulation over formulae
* Each logic has a particular language which defines the symbols available for a formula: *true*, *false*, *P*, *Q*, *R*, $\neg$, $\forall$... 
* Rules define which formulae are well-formed e.g. in propositional logic ($\implies$ *P*) is not well-formed

# Symbolic Logic - Propositional Logic

* A *proposition* is a declarative sentence with a truth value *true* or *false*
* In symbolic logic denote a proposition by letters like *P*, *Q*, *R*
* Symbols *P*, *Q*, *R* called atomic formulae, 
* Symbols $\land$, $\lor$, $\neg$, $\implies$called logical connectives.
* Computer programming languages have symbols like `||` or `&&` which represent logical connectives
* Atomic formulae and logical connectives form compound formulae

In [3]:
let P = 6 > 5 // atomic formula
let Q = 400 / 10 < 30 // atomic formula
P, Q, not Q, P || Q, P && Q, P ==> Q // compound formula

(true, false, true, true, false, false)

# Symbolic Logic - Boolean Satisfiability
* An *interpretation* is just an assignment of truth values to atomic formulae in a well-formed formula
* Truth-tables enumerate interpretations of a well-formed formula
![tt](truthtablet.png)
* A *model* is an interpretation where a Formula is true according to the rules of a logic
* A formula is *satisfiable* if it has a model i.e. an interpretation that makes it true
* For a formula with *n* atomic formulae there are 2<sup>n</sup> possible interpretations

# Symbolic Logic - Boolean Satisfiability
* e.g. A digital logic circuit with 5 inputs has 2<sup>5</sup> = 32 states 
![circuit](circuit.jpg)
* Satisfiability easy to check for small formulas, but explodes with exponential complexity for large formulae
* A digital logic circuit wth 100 inputs has 1267650600228229401496703205376 states

# Symbolic Logic -  The Boolean Satisfiability Problem (SAT)
* **Is there an interpretation of a propositional logic formula *F* that satisfies *F*?**
* e.g What truth values of *P*, *Q*, *R* make the formula (*P* $\implies$ *Q* $\lor$ *R*) true? 
* Equally important: is a formula *unsatisfiable*? e.g. *P* $\land$ $\neg$ *P* is unsatisfiable
* SAT problem is *decidable*, worst-case algorithm can just check truth-tables
* Many open-source SAT solvers: MiniSat, PicoSat et.al

# Symbolic Logic - First-order Logic
* Consider a set which will be the domain or universe of discourse and constants or elements belonging to the universe of disourse
* The language of first-order logic includes quantifiers, variables, functions, predicates
* e.g. $\forall (x, y): P(x) = P(y) \implies x = y$ is a formula in first-order logic
* Many mathematical theories can be formalized using first-order logic

# Satisfiability Modulo Theories (SMT)

* First-order satisfiability is not in *general* decidable
* However, many theories or fragments of theories are decidable
* real numbers, integers, theories data structures such as arrays, bit vectors, and strings
* In certain theory fragments quantifiers can be eliminated or the problem reduced to SAT
* Consider first-order satisfiability 'modulo' (within) a particular theory

# Satisfiability Modulo Theories (SMT)
>The defining problem of Satisfiability Modulo Theories (SMT) is checking whether a given logical formula F is satisfiable in the context of some background theory which constraints the interpretation of the
symbols used in F
* e.g. $a + b > 3$ and $a < 0$ and $b > 0$ is satisfiable using $a = -1$ and $b = 5$
* SMT solvers attempt to decide in a reasonable amount of time if a first-order formula is satisfiable or unsatisfiable in the context of some background theory 
* Many open-source solvers available today e.g Yices, CVC4, Z3

# The Z3 Solver
>Z3 is a state-of-the-art SMT solver from Microsoft Research. It integrates a host of theory solvers in an
expressive and efficient combination.
* One of the most popular SMT solvers today used in or supported by a large number of applications in diverse fields
* Many, many use cases in mathematical analysis, optimization, program verification, computer security e.g. verifying firewall rules: https://github.com/Z3Prover/FirewallChecker

## The Z3 Solver
* Native language is a Lisp-like language SMTLIB2
* Interfaces for many languages: C++, Python, OCaml,.NET,... 
* https://github.com/Z3Prover/z3/tree/master/src/api
* .NET API Z3 ships with tends to be low-level and oriented towards imperative code

````csharp
//Z3 C# interface example code

FPSort double_sort = ctx.MkFPSort(11, 53);
FPRMSort rm_sort = ctx.MkFPRoundingModeSort();

FPRMExpr rm = (FPRMExpr)ctx.MkConst(ctx.MkSymbol("rm"), rm_sort);
    BitVecExpr x = (BitVecExpr)ctx.MkConst(ctx.MkSymbol("x"), ctx.MkBitVecSort(64));

FPExpr y = (FPExpr)ctx.MkConst(ctx.MkSymbol("y"), double_sort);
FPExpr fp_val = ctx.MkFP(42, double_sort);

BoolExpr c1 = ctx.MkEq(y, fp_val);
BoolExpr c2 = ctx.MkEq(x, ctx.MkFPToBV(rm, y, 64, false));
BoolExpr c3 = ctx.MkEq(x, ctx.MkBV(42, 64));
BoolExpr c4 = ctx.MkEq(ctx.MkNumeral(42, ctx.RealSort), 
                       ctx.MkFPToReal(fp_val));

BoolExpr c5 = ctx.MkAnd(c1, c2, c3, c4);
````

# How can we use Z3 in a functional programming language like F#? 

# F# Quotations

# F# Quotations

* https://docs.microsoft.com/en-us/dotnet/fsharp/language-reference/code-quotations
* Similar to OCaml quotations
* F# code delimited by `<@ @>` is interpreted as a syntactic structure
* Primary vehicle for F# meta-programming
* Same compiler and IDE features available for writing ordinary F# source code

# F# Quotations - Examples

In [4]:
// Ordinary F# code
4 + 5 

9

In [5]:
// F# code quotation
<@ 4 + 5 @>

Call (None, op_Addition, [Value (4), Value (5)])

In [6]:
(4 + 5).GetType()

System.Int32

In [7]:
(<@ 4 + 5 @>).GetType()

Microsoft.FSharp.Quotations.FSharpExpr`1[System.Int32]

In [8]:
let f a b = a > b || a - b = 1
<@ f @>

Lambda (a, Lambda (b, Call (None, f, [a, b])))

In [9]:
<@ f @>.GetType()

Microsoft.FSharp.Quotations.FSharpExpr`1[Microsoft.FSharp.Core.FSharpFunc`2[System.Int32,Microsoft.FSharp.Core.FSharpFunc`2[System.Int32,System.Boolean]]]

# F# Code Quotations
* Have type `Expr<'a>` or `Expr`
* Examples above have type `Expr<int>` or `Expr<bool>`
* Instances of `Expr<'a>` or `Expr` treat code as purely symbolic
* Use pattern-matching
* F# code inside quotations can be manipulated and synthesized
* Primary vehicle for deep embedding of DSLs in F#

In [10]:
let j = <@ 5 @>
<@ 3 + 4 + %j @>

Call (None, op_Addition,
      [Call (None, op_Addition, [Value (3), Value (4)]), Value (5)])

In [11]:
open FSharp.Quotations.Patterns
open FSharp.Quotations.DerivedPatterns

match <@ 3 + 6 + %j @> with
| SpecificCall <@@ (+) @@> (None,_,l::r::[]) -> Some r
| _ -> None

Some Value (5)

In [12]:
// Most F# environments support editing code quotations with the same features as regular code
<@ 3 + 4. @>

The type 'float' does not match the type 'int'
The type 'float' does not match the type 'int'

In [13]:
// Functions have both ordinary and symbolic form
[<Formula>] 
let g x y = x * y > x / y //Boolean-valued function

g 4 5

true

In [14]:
expand <@ g @>

Lambda (x,
        Lambda (y,
                Call (None, op_GreaterThan,
                      [Call (None, op_Multiply, [x, y]),
                       Call (None, op_Division, [x, y])])))

# Integrating Z3 with F#

# Integrating Z3 with F#
* Need to represent Z3's symbolic expressions and sorts
* Ideally use pattern matching for code translation and generation
* One option is to define custom types, operators e.g Python's `IntVar`...

## Integrating Z3 with F#
  
  One approach is to use custom types like the Python Z3 library:
  ````python
  Z = IntSort()
  f = Function('f', Z, Z)
  x, y, z = Ints('x y z')
  A = Array('A', Z, Z)
  fml = Implies(x + 2 == y, f(Store(A, x, 3)[y - 2]) == f(y - x + 1))
````

## Integrating Z3 with F#
* ...or we can use built-in `Expr<'t>` from F# code quotations
* Natural mapping of numeric types e.g.: `3` -> `<@ 3 @>`
* Full compiler and IDE support: type checking, syntax highlighting for quotations
* Comprehensive library of patterns
* Easy to compose your own patterns

# Using Z3 with F# code quotations

* Need a way to represent symbolic variables of a particular *sort* (roughly similar to type)

* Need to represent common arithmetic and logical operations: `+`, `-`, `*`, $\land$, $\lor$, $\forall$ etc.

* Need to represent common mathematical forms e.g. systems of equations, sequences

# Translating F# quotations to Z3 logical expressions
* Define a function for creating symbolic variables
* Z3 sorts represented by F# types
* Use lists of `Expr` to define constraints or formula parts to be solved for satisfiability by Z3

In [1]:
open FSharp.Quotations
// Use Expr.Var expression for symbolic variable with name
let symbolic_var<'t> n = 
    let v = Expr.Var(Var(n, typeof<'t>)) in <@ %%v:'t @>

// Alias float type as real
type real = float

// Map sorts to ordinary types
let intvar x = symbolic_var<int> x
let realvar x = symbolic_var<real> x

let x,y = intvar "x", realvar "y"
<@ %x + %x > %x @> // Ok
//<@ %x + %y @> // Doesn't typecheck

The type 'real' does not match the type 'int'
The type 'real' does not match the type 'int'

````fsharp
// Need a type for rational numbers
// Inspired by: https://github.com/mathnet/mathnet-numerics/blob/master/src/FSharp/BigRational.fs
open MathNet.Numerics
[<CustomEquality; CustomComparison>]
type Rational = 
    struct 
        val Numerator: BigInteger
        val Denominator: BigInteger
        new(p:BigInteger, q:BigInteger) = {Numerator = p; Denominator = q}
        new(p:BigInteger) = {Numerator = p; Denominator = BigInteger.One}
        new(p:int, q:int) = {Numerator = BigInteger p; Denominator = BigInteger q}
        //...
type rat = Rational // alias
let ratvar = symbolic_var<rat>
````

````fsharp
// Check satisfiability of list of Boolean conditions
let internal check_sat_model (s:Z3Solver) (a: Expr<bool list>) = 
        let sol = a |> expand_list |> List.map (create_bool_expr s) |> s.Check 
        match sol with
        | Status.SATISFIABLE -> Some (s.Model())
        | _ -> None
        
let check_sat (s:Z3Solver) a = (Option.isSome <| check_sat_model s a)
````

# Using Z3 with F# code quotations - First examples

In [7]:
// Open Z3 module and create instance of solver
open Z3
let z3 = new Z3Solver()

// Declare 2 symbolic variables. Variables can be reused.
let r, s = intvar "r", intvar "s"

// Check satisfiability of simple integer equation system
check_sat z3 <@[ 
    %r * %s = 6
    %r - %s = 1
]@> // Use lists to represent equation system

true

In [16]:
let p, q = boolvar "p", boolvar "q"
check_sat z3 <@[ %p |&| not %p ]@>

false

# Propositional and first-order logic formula satisfiability

In [28]:
(*
(declare-preds ((p1) (p2) (p3) (p4) (p5)))
(assert (=> p1 p2))
(assert (=> p1 p3))
(assert (=> p1 p4))
(assert (not p2)
*)

let p1,p2,p3,p4,p5 = boolvar "p1", boolvar "p2", boolvar "p3", boolvar "p4", boolvar "p5"
<@[
    %p1 ==> %p2
    %p1 ==> %p3
    %p1 ==> %p4
    not %p2
]@> |> get_bool_var_model z3

Some [("p2", false); ("p1", false)]

Consider the set formula $\forall A: r \in A \lor r \notin A$

In [9]:
let A = setvar<int> "A" //Create a symbolic variable with Set sort
let B = setvar<real> "B"
<@ A |+| A @>
// |?| is the set membership operator
// 
//<@[ forall' %r (%r |?| %A ||| (not (%r |?| %A))) ]@> |> check_sat z3

The type 'Quotations.Expr<Set<int>>' is not compatible with the type 'ISet<obj>'
The type 'Quotations.Expr<Set<int>>' is not compatible with the type 'ISet<obj>'

# Uninterpreted functions

In [15]:
// Uninterpreted
let func<'s, 't> = fun (_:'s) -> Unchecked.defaultof<'t>

let h = func<int, bool>
let hh = func<int, int>

let m = check_sat_model z3 <@[ (hh(hh 4)) <> (hh 4) ]@>
m.[z3, <@ (hh 4) @>]

The field, constructor or member 'Item' is not defined.

In [31]:
m.Value.[z3,  <@ hh 4 @> ]

FuncResult [2 -> 3, else -> 2]

# Arithmetic and Logical puzzles

![dgm](dgm.png)

In [19]:
let dog, cat, mouse = intvar3 "dog" "cat" "mouse"
let puzzle1 = <@[%dog >= 1; %cat >= 1; %dog + %cat + %mouse = 100; 1500 * %dog + 100 * %cat + 25 * %mouse = 10000 ]@>
check_sat z3 puzzle1

true

In [20]:
get_int_var_model z3 puzzle1

Some [("cat", 41); ("mouse", 56); ("dog", 3)]

![xkcd](https://imgs.xkcd.com/comics/np_complete.png)

In [21]:
let a, b,c ,d = intvar4 "a" "b" "c" "d"
let e, f = intvar2 "e" "f"
let puzzle2 = <@[ 215 * %a + 275 * %b + 335 * %c + 355 * %d + 420 * %e + 580 * %f = 1505 ]@>
check_sat z3 puzzle2

true

In [22]:
check_sat_model z3 puzzle2

Some
  (define-fun f () Int
  2)
(define-fun b () Int
  1)
(define-fun a () Int
  4)
(define-fun d () Int
  19)
(define-fun c () Int
  (- 25))
(define-fun e () Int
  2)

# Optimization and Constraints

In [24]:
open LP
let x = ratvar "x"
let y = ratvar "y"

let fo = max false  <@[ 6* %x + 5 * %y = 45Q; %x > 0Q; %y > 0Q; ]@> <@ %x * %x * %y @>

match fo with
| Some (RatLPSol r) -> r
| _  -> []

[("y", 4495/4096); ("x", 161845/24576)]